In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from qdrant_client.http import models
import uuid
from tqdm import tqdm
import json
from typing import List, Dict, Any
import re

In [ ]:
class LegalDocumentVectorDB:
    def __init__(self, qdrant_url: str, api_key: str, collection_name: str = "hukuki kararlar"):
        """
        Hukuki belgeler için vector database sınıfı
        
        Args:
            qdrant_url: Qdrant sunucu URL'i
            api_key: Qdrant API anahtarı  
            collection_name: Collection adı
        """
        self.client = QdrantClient(
            url=qdrant_url,
            api_key=api_key,
            timeout=60
        )
        self.collection_name = collection_name
        
        # Türkçe için optimize edilmiş multilingual model
        print("Sentence Transformer modeli yükleniyor...")
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        print("Model başarıyla yüklendi!")
        
        # Vector boyutunu al
        self.vector_size = self.model.get_sentence_embedding_dimension()
        print(f"Vector boyutu: {self.vector_size}")

    def create_collection(self, recreate: bool = False):
        """Collection oluştur"""
        try:
            if recreate:
                self.client.delete_collection(collection_name=self.collection_name)
                print(f"Eski collection '{self.collection_name}' silindi.")
        except:
            pass
        
        try:
            self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(
                    size=self.vector_size, 
                    distance=Distance.COSINE
                )
            )
            print(f"Collection '{self.collection_name}' oluşturuldu.")
        except Exception as e:
            if "already exists" in str(e).lower():
                print(f"Collection '{self.collection_name}' zaten mevcut.")
            else:
                raise e

    def clean_text(self, text: str) -> str:
        """Metni temizle"""
        if pd.isna(text):
            return ""
        
        # Encoding sorunlarını düzelt
        text = str(text)
        replacements = {
            'Ã¤': 'ä', 'Ã¶': 'ö', 'Ã¼': 'ü', 'ÃŸ': 'ß',
            'Ã‡': 'Ç', 'Ä±': 'ı', 'Ä°': 'İ', 'ÅŸ': 'ş',
            'Ä\x9f': 'ğ', 'Ã§': 'ç', 'Ã¶': 'ö', 'Ã¼': 'ü'
        }
        
        for old, new in replacements.items():
            text = text.replace(old, new)
        
        # Fazla boşlukları temizle
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text

    def process_csv(self, csv_path: str) -> pd.DataFrame:
        """CSV dosyasını işle"""
        print(f"CSV dosyası okunuyor: {csv_path}")
        
        # Dosya varlığını kontrol et
        import os
        if not os.path.exists(csv_path):
            print(f"❌ HATA: Dosya bulunamadı: {csv_path}")
            return None
        
        try:
            # Encoding denemesi
            try:
                df = pd.read_csv(csv_path, encoding='utf-8')
                print("✅ UTF-8 encoding ile başarıyla okundu")
            except UnicodeDecodeError:
                df = pd.read_csv(csv_path, encoding='latin-1')
                print("✅ Latin-1 encoding ile başarıyla okundu")
        except Exception as e:
            print(f"❌ CSV okuma hatası: {e}")
            return None
        
        print(f"Toplam satır sayısı: {len(df)}")
        print(f"Sütunlar: {df.columns.tolist()}")
        
        # Boş chunk_text'leri filtrele
        initial_count = len(df)
        df = df.dropna(subset=['chunk_text'])
        df = df[df['chunk_text'].str.strip() != '']
        final_count = len(df)
        
        print(f"Boş metin filtrelemesi: {initial_count} -> {final_count}")
        
        # Metinleri temizle
        df['chunk_text_clean'] = df['chunk_text'].apply(self.clean_text)
        
        return df

    def create_embeddings_batch(self, texts: List[str], batch_size: int = 32) -> List[List[float]]:
        """Metinleri batch halinde embedding'e çevir"""
        embeddings = []
        
        for i in tqdm(range(0, len(texts), batch_size), desc="Embedding oluşturuluyor"):
            batch = texts[i:i+batch_size]
            batch_embeddings = self.model.encode(
                batch, 
                convert_to_numpy=True,
                show_progress_bar=False,
                normalize_embeddings=True  # Cosine similarity için normalize et
            )
            embeddings.extend(batch_embeddings.tolist())
            print(embeddings)
        
        return embeddings

    def upload_to_qdrant(self, df: pd.DataFrame, batch_size: int = 100):
        """DataFrame'i Qdrant'a yükle"""
        print("Qdrant'a yükleme başlıyor...")
        
        # Embeddings oluştur
        texts = df['chunk_text_clean'].tolist()
        embeddings = self.create_embeddings_batch(texts, batch_size=32)
        
        # Points oluştur
        points = []
        for idx, (_, row) in enumerate(df.iterrows()):
            payload = {
                "document_id": str(row['_id']),
                "location": str(row['location']),
                "esas_no": str(row['esasNo']),
                "karar_no": str(row['kararNo']),
                "dates": str(row['extractedDates']),
                "daire": str(row['daire']),
                "mahkeme": str(row['mahkeme']),
                "karar_turu": str(row['karar_turu']),
                "chunk_id": str(row['chunk_id']),
                "chunk_index": int(row['chunk_index']),
                "total_chunks": int(row['total_chunks']),
                "chunk_text": str(row['chunk_text_clean']),
                "chunk_length": int(row['chunk_length'])
            }
            
            point = PointStruct(
                id=str(uuid.uuid4()),
                vector=embeddings[idx],
                payload=payload
            )
            points.append(point)
        
        # Batch halinde yükle
        for i in tqdm(range(0, len(points), batch_size), desc="Qdrant'a yükleniyor"):
            batch_points = points[i:i+batch_size]
            
            try:
                self.client.upsert(
                    collection_name=self.collection_name,
                    points=batch_points
                )
            except Exception as e:
                print(f"Yükleme hatası (batch {i//batch_size + 1}): {e}")
                continue
        
        print(f"Toplam {len(points)} doküman başarıyla yüklendi!")

    def search(self, query: str, limit: int = 5, score_threshold: float = 0.5) -> List[Dict[str, Any]]:
        """Semantik arama yap"""
        print(f"Arama yapılıyor: '{query}'")
        
        # Query embedding'i oluştur
        query_embedding = self.model.encode(
            [query], 
            convert_to_numpy=True,
            normalize_embeddings=True
        )[0].tolist()
        
        # Arama yap (query_points kullan)
        search_results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding,
            limit=limit,
            score_threshold=score_threshold
        )
        
        results = []
        for result in search_results.points:
            results.append({
                "score": result.score,
                "payload": result.payload
            })
        
        return results

    def advanced_search(self, query: str, filters: Dict = None, limit: int = 5) -> List[Dict[str, Any]]:
        """Gelişmiş filtreleme ile arama"""
        query_embedding = self.model.encode(
            [query], 
            convert_to_numpy=True,
            normalize_embeddings=True
        )[0].tolist()
        
        # Filter oluştur
        filter_conditions = None
        if filters:
            conditions = []
            
            if 'daire' in filters:
                conditions.append(models.FieldCondition(
                    key="daire",
                    match=models.MatchValue(value=filters['daire'])
                ))
            
            if 'karar_turu' in filters:
                conditions.append(models.FieldCondition(
                    key="karar_turu", 
                    match=models.MatchValue(value=filters['karar_turu'])
                ))
            
            if 'year' in filters:
                conditions.append(models.FieldCondition(
                    key="dates",
                    match=models.MatchText(text=str(filters['year']))
                ))
            
            if conditions:
                filter_conditions = models.Filter(must=conditions)
        
        # query_points kullan
        search_results = self.client.query_points(
            collection_name=self.collection_name,
            query=query_embedding,
            query_filter=filter_conditions,
            limit=limit
        )
        
        results = []
        for result in search_results.points:
            results.append({
                "score": result.score,
                "payload": result.payload
            })
        
        return results

    def get_collection_info(self):
        """Collection bilgilerini getir"""
        try:
            info = self.client.get_collection(collection_name=self.collection_name)
            result = {
                "status": str(info.status),
                "vectors_count": info.vectors_count if hasattr(info, 'vectors_count') else 0,
            }
            
            # Mevcut attributeları kontrol et ve ekle
            if hasattr(info, 'segments_count'):
                result["segments_count"] = info.segments_count
            if hasattr(info, 'indexed_vectors_count'):
                result["indexed_vectors_count"] = info.indexed_vectors_count
            if hasattr(info, 'points_count'):
                result["points_count"] = info.points_count
                
            return result
        except Exception as e:
            return {"error": str(e)}

In [ ]:
class RetrievalEvaluator:
    def __init__(self, db: LegalDocumentVectorDB, queries_file: str):
        self.db = db
        with open(queries_file, "r", encoding="utf-8") as f:
            self.queries = json.load(f)

    def evaluate(self, limit: int = 5) -> pd.DataFrame:
        """Arama kalitesini ölç"""
        results_summary = []

        for q in self.queries:
            query_text = q["query"]
            expected_keywords = q.get("expected_keywords", [])

            res = self.db.search(query_text, limit=limit)

            # Basit keyword match skoru
            hits = 0
            for r in res:
                text = r["payload"]["chunk_text"].lower()
                if any(kw.lower() in text for kw in expected_keywords):
                    hits += 1

            results_summary.append({
                "query": query_text,
                "expected_keywords": expected_keywords,
                "retrieved": len(res),
                "hits": hits,
                "hit_rate": round(hits / max(1, len(res)), 3),
                "avg_score": round(np.mean([r["score"] for r in res]), 3) if res else 0
            })

        df = pd.DataFrame(results_summary)
        return df

    def save_results(self, df: pd.DataFrame, filename: str = "eval_results.csv"):
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"📊 Sonuçlar kaydedildi: {filename}")


In [ ]:
class InteractiveLegalSearch:
        def __init__(self, qdrant_url: str, api_key: str,collection_name: str):
            self.db = LegalDocumentVectorDB(qdrant_url, api_key,collection_name)
        
        def setup_database(self, csv_file: str):
            """Database'i kur"""
            print("Database kurulumu başlıyor...")
            self.db.create_collection(recreate=True)
            
            df = self.db.process_csv(csv_file)
            if df is not None:
                self.db.upload_to_qdrant(df)
                print("Database kurulumu tamamlandı!")
                return True
            return False
        
        def interactive_search(self):
            """Interaktif arama"""
            while True:
                print("\n" + "="*50)
                print("HUKUKİ KARAR ARAMA SİSTEMİ")
                print("="*50)
                
                query = input("\nAramak istediğiniz metni girin (çıkmak için 'q'): ")
                if query.lower() == 'q':
                    break
                
                try:
                    limit = int(input("Kaç sonuç gösterilsin? (varsayılan 5): ") or "5")
                except:
                    limit = 5
                
                results = self.db.search(query, limit=limit)
                
                if not results:
                    print("❌ Sonuç bulunamadı.")
                    continue
                
                print(f"\n🔍 '{query}' için {len(results)} sonuç bulundu:")
                print("-" * 50)
                
                for i, result in enumerate(results, 1):
                    payload = result['payload']
                    print(f"\n📄 {i}. SONUÇ (Benzerlik: {result['score']:.3f})")
                    print(f"   📂 Daire: {payload['daire']}")
                    print(f"   📋 Esas No: {payload['esas_no']}")
                    print(f"   ⚖️ Karar No: {payload['karar_no']}")
                    print(f"   🏛️ Mahkeme: {payload['mahkeme']}")
                    print(f"   📅 Tarihler: {payload['dates']}")
                    print(f"   📝 Chunk: {payload['chunk_index']}/{payload['total_chunks']}")
                    print(f"   📄 Metin Önizleme:")
                    print(f"      {payload['chunk_text'][:300]}...")
                    print("-" * 30)

In [9]:
if __name__ == "__main__":
    mode = input("Mod seç (1=Interaktif Arama, 2=Evaluation): ")

    searcher = InteractiveLegalSearch(
        qdrant_url="https://qdrant.adalet.gov.tr:443",
        api_key="kMy0juEwUcsLjKDjWTPUAWTYlYpR3kjh",
        collection_name="hukuki_kararlar"
    )

    if mode == "1":
        searcher.interactive_search()
    elif mode == "2":
        evaluator = RetrievalEvaluator(searcher.db, "queries.json")
        df = evaluator.evaluate(limit=5)
        print("\n=== Evaluation Sonuçları ===")
        print(df)
        evaluator.save_results(df, "eval_results.csv")
    else:
        print("❌ Geçersiz seçim!")


/tmp/ipykernel_3717247/3988472342.py:11: UserWarning: Qdrant client version 1.15.1 is incompatible with server version 1.12.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  self.client = QdrantClient(


Sentence Transformer modeli yükleniyor...
Model başarıyla yüklendi!
Vector boyutu: 384
Arama yapılıyor: 'Yargıtay 6. Hukuk Dairesi kira sözleşmesiyle ilgili hangi kararı verdi?'
Arama yapılıyor: 'Müteselsil borç kavramı hangi kararda nasıl yorumlandı?'
Arama yapılıyor: 'Yargıtay 13. Hukuk Dairesi'nin karar gerekçesi nedir?'
Arama yapılıyor: 'Tazminat davalarında hangi esaslara göre hüküm verilmektedir?'
Arama yapılıyor: 'Hangi davada taraflar arasında sözleşme feshi söz konusuydu?'
Arama yapılıyor: 'Yargıtay hangi kararda usul hatası nedeniyle bozma kararı verdi?'
Arama yapılıyor: 'Mahkeme kararında davacının talebi ne şekilde değerlendirilmiştir?'
Arama yapılıyor: 'İş mahkemesi kararlarında işçi alacakları nasıl hükme bağlanıyor?'
Arama yapılıyor: 'Hangi kararda icra takibine ilişkin hüküm yer alıyor?'
Arama yapılıyor: 'Borçlu temerrüdü hangi içtihatta nasıl açıklanmıştır?'

=== Evaluation Sonuçları ===
                                               query  \
0  Yargıtay 6. Hukuk Daire